In [1]:
# 初始化
%load_ext autoreload
%autoreload 2
import sys
import os
os.chdir('E:\GitHub\QA-abstract-and-reasoning')
sys.path.append('E:\GitHub\QA-abstract-and-reasoning')

E:\software\Anaconda3\envs\tf2.0\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
import tensorflow as tf
from pgn.layers import BahdanauAttention, Encoder, Pointer
from pgn.batcher import batcher
from pgn.model import PGN
from utils.saveLoader import load_embedding_matrix
from utils.saveLoader import Vocab
from utils.config import VOCAB_PAD
from utils.config_gpu import config_gpu
config_gpu()

1 Physical GPUs, 1 Logical GPUs


In [3]:
%run utils/params.py
# 产生输入数据
vocab = Vocab(VOCAB_PAD)
ds =iter(batcher(vocab, params))
enc_data, dec_data = next(ds)

In [4]:
config_gpu()

1 Physical GPUs, 1 Logical GPUs


In [7]:


print("Building the model ...")

model = PGN(params)

print("Creating the vocab ...")
vocab = Vocab(params["vocab_path"], params["vocab_size"])
# ds = batcher(vocab, params)

print("Creating the checkpoint manager")
checkpoint = tf.train.Checkpoint(Seq2Seq=model)

checkpoint_manager = tf.train.CheckpointManager(checkpoint, PGN_CKPT, max_to_keep=5)

# checkpoint_manager = tf.train.CheckpointManager(checkpoint, TEMP_CKPT, max_to_keep=5)
# temp_ckpt = os.path.join(TEMP_CKPT, "ckpt-5")
# checkpoint.restore(temp_ckpt)

checkpoint.restore(checkpoint_manager.latest_checkpoint)
if checkpoint_manager.latest_checkpoint:
    print("Restored from {}".format(checkpoint_manager.latest_checkpoint))
else:
    print("Initializing from scratch.")
print("Model restored")

1 Physical GPUs, 1 Logical GPUs
Building the model ...
Creating the vocab ...
Creating the checkpoint manager
Restored from E:\GitHub\QA-abstract-and-reasoning\data\checkpoints\pgn_checkpoints\ckpt-1
Model restored


# oov index的decoder

In [21]:
from pgn.test_helper import output_id_to_word

In [22]:
vocab.count

32233

In [23]:
article_oovs = enc_data["article_oovs"][3]
article_oovs

<tf.Tensor: id=233, shape=(4,), dtype=string, numpy=array([b'\xe5\x87\xba\xe8\x84\xb8', b'', b'', b''], dtype=object)>

In [25]:
idx = 32233
output_id_to_word(idx, vocab, article_oovs)

'出脸'

In [31]:
dec_input = tf.constant([vocab.word2id[vocab.START_DECODING]] * 64)

In [34]:
dec_input1 = []

In [35]:
for i in range(10):
    dec_input1.append(i)

In [39]:
dec_input

<tf.Tensor: id=242, shape=(64,), dtype=int32, numpy=
array([32229, 32229, 32229, 32229, 32229, 32229, 32229, 32229, 32229,
       32229, 32229, 32229, 32229, 32229, 32229, 32229, 32229, 32229,
       32229, 32229, 32229, 32229, 32229, 32229, 32229, 32229, 32229,
       32229, 32229, 32229, 32229, 32229, 32229, 32229, 32229, 32229,
       32229, 32229, 32229, 32229, 32229, 32229, 32229, 32229, 32229,
       32229, 32229, 32229, 32229, 32229, 32229, 32229, 32229, 32229,
       32229, 32229, 32229, 32229, 32229, 32229, 32229, 32229, 32229,
       32229])>

In [40]:
vocab.word2id[vocab.UNKNOWN_TOKEN]

32232

In [41]:
a = "123456789"

In [45]:
a.index("56789")

4

In [28]:
vocab.id2word[32233]

KeyError: 32233